Short-Term Memory

In [1]:
from langchain_huggingface import HuggingFaceEndpoint, ChatHuggingFace
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
llm = HuggingFaceEndpoint(
    repo_id= "meta-llama/Llama-3.2-1B-Instruct",
    task= "text-generation")

chat = ChatHuggingFace(
    llm = llm)

In [5]:
#Add custom chat history to give to the model
conversation_history = [
    {"role": "system", "content": "You are a helpful tutor."},
    {"role": "user", "content": "My name is Narendra Modi"}]

In [8]:
response1 = chat.invoke(input= conversation_history)

In [9]:
response1.content

"Mr. Narendra Modi, it's an honor to speak with you. As a tutor, I'm here to help you with any questions or topics you'd like to discuss. What's on your mind? Are you preparing for a specific exam, working on a project, or just want to chat about a particular subject?"

In [10]:
#Adding the latest interaction to the conversation history and adding a propmt
conversation_history.append({"role":"assistant", "content":response1.content})
conversation_history.append({"role":"user", "content":"What is my name?"})

In [ ]:
#Checking if the model remembers the name from the previous interaction
response2 = chat.invoke(input= conversation_history)
response2.content

'Your name is Narendra Modi.'

Long Term Memory Storage using VectorDB (ChromaDB here)

In [17]:
from langchain_chroma import Chroma
from langchain_huggingface import HuggingFaceEndpointEmbeddings
from langchain_core.documents import Document

In [18]:
embed = HuggingFaceEndpointEmbeddings(
    model = "sentence-transformers/all-mpnet-base-v2", 
    task="feature-extraction")

In [20]:
#Creating a vector store
docs = [Document(page_content="Narendra Modi is the current Prime Minister of India."),
        Document(page_content="Narendra Modi sworn for the 3rd time as PM on 9th June 2024"),
        Document(page_content="India is hosting 18th BRICS summit in 2026.")]

db = Chroma.from_documents(documents= docs, embedding= embed, collection_name="long_term_memory_db")

In [25]:
#Query Function
def recall(query):
    retriever = db.as_retriever(search_type="similarity", search_kwargs={"k":2})
    results = retriever.invoke(query)
    return [r.page_content for r in results]

In [ ]:
#Should give all the relevant documents for this query
print(recall("Who is the current Prime Minister of India?"))

['Narendra Modi is the current Prime Minister of India.', 'Narendra Modi sworn for the 3rd time as PM on 9th June 2024']
